In [2]:
import time
import numpy as np
import pymahjong as pm
from pymahjong import MahjongEnv


env = MahjongEnv()
num_games = 1


In [3]:
UNICODE_TILES = """
    🀇 🀈 🀉 🀊 🀋 🀌 🀍 🀎 🀏 
    🀙 🀚 🀛 🀜 🀝 🀞 🀟 🀠 🀡
    🀐 🀑 🀒 🀓 🀔 🀕 🀖 🀗 🀘
    🀀 🀁 🀂 🀃
    🀆 🀅 🀄
    🀋 🀝 🀔
""".split()

ACTIONs = ["摸牌", "摸杠牌", "手切", "摸切", "吃L", "吃M", "吃R", "碰", "暗杠", "加杠", "摸切立直", "手切立直", "立直通过", "关联自家", "关联下家", "关联对家", "关联上家"] 


In [24]:
def render_global_info(global_info):
    explains = ["局数", "最终局", "本场数", "场供数", "自风牌", "场风牌", "自家点数", "下家点数", "对面点数", "上家点数", "自家一发", "下家一发", "对面一发", "上家一发", "剩余牌数"]
    for i, e in enumerate(explains):
        print(e + ":" + str(global_info[i]))

In [19]:
def render_encoding_record(record):
    a = np.argwhere(np.array(record)).reshape([-1])
    action_strs = []
    for i in a:
        if i < 37:
            action_strs.append(UNICODE_TILES[i])
        else:
            action_strs.append(ACTIONS[i - 37])
    print(", ".join(action_strs))
    

In [4]:
def render_encoding_self_info(self_info):
    # 0-3
    
    hands = list(np.argwhere(self_info[:, 0]).reshape([-1])) + list(np.argwhere(self_info[:, 1]).reshape([-1])) + list(np.argwhere(self_info[:, 2]).reshape([-1])) + list(np.argwhere(self_info[:, 3]).reshape([-1]))
    hands.sort()
    print("-------- Hand -------------")
    print("".join([UNICODE_TILES[i] for i in hands]))
    
    hand_akas = list(np.argwhere(self_info[:, 6]).reshape([-1])) 
    print("-------- Aka -------------")
    print("".join([UNICODE_TILES[i] for i in hand_akas]))
    
    
    doras = list(np.argwhere(self_info[:, 4]).reshape([-1]))
    print("-------- Dora -------------")
    print("".join([UNICODE_TILES[i] for i in doras]))
    
    dora_indicators = list(np.argwhere(self_info[:, 5]).reshape([-1]))
    print("-------- Dora Indicator -------------")
    print("".join([UNICODE_TILES[i] for i in dora_indicators]))
    
    changfengs = list(np.argwhere(self_info[:, 7]).reshape([-1]))
    print("-------- Game Wind -------------")
    print("".join([UNICODE_TILES[i] for i in changfengs]))
    
    zifengs = list(np.argwhere(self_info[:, 8]).reshape([-1]))
    print("-------- Self Wind -------------")
    print("".join([UNICODE_TILES[i] for i in zifengs]))
    
    
    zifengs = list(np.argwhere(self_info[:, 9]).reshape([-1]))
    print("-------- Tsumo Tiles -------------")
    print("".join([UNICODE_TILES[i] for i in zifengs]))
    
    tmp = list(np.argwhere(self_info[:, 10]).reshape([-1]))
    print("-------- Self Discarded -------------")
    print("".join([UNICODE_TILES[i] for i in tmp]))
    
    tmp = list(np.argwhere(self_info[:, 11]).reshape([-1]))
    print("-------- Next Discarded -------------")
    print("".join([UNICODE_TILES[i] for i in tmp]))
    
    tmp = list(np.argwhere(self_info[:, 12]).reshape([-1]))
    print("-------- Opposite Discarded -------------")
    print("".join([UNICODE_TILES[i] for i in tmp]))
    
    tmp = list(np.argwhere(self_info[:, 13]).reshape([-1]))
    print("-------- Previous Discarded -------------")
    print("".join([UNICODE_TILES[i] for i in tmp]))
    
    
    tmp = list(np.argwhere(self_info[:, 14]).reshape([-1])) + list(np.argwhere(self_info[:, 15]).reshape([-1])) + list(np.argwhere(self_info[:, 16]).reshape([-1])) + list(np.argwhere(self_info[:, 17]).reshape([-1]))
    tmp.sort()
    print("-------- Disclosed Tiles -------------")
    print("".join([UNICODE_TILES[i] for i in tmp]))
    

In [28]:


start_time = time.time()
game = 0
success_games = 0

winds = ["east", "south", "west", "north"]

while game < num_games:
    
    try:
        
        # env.reset(oya=game % 4, game_wind=winds[game % 3], debug_mode=1)
        env.reset(oya=2, game_wind='south', debug_mode=1)

        # encoder
        te = pm.TableEncoder(env.t)
        te.init()
        
        te.update()

        # print(np.array(te.self_infos[0]).reshape([18, 34]).swapaxes(0, 1))
        for i in range(4):
            print("东风局，庄家是 {}, player {} global info".format(game % 4, i), te.global_infos[i])
            
        while not env.is_over():

            curr_player_id = env.get_curr_player_id()

            # --------- get decision information -------------

            valid_actions_mask = env.get_valid_actions(nhot=True)
            executor_obs = env.get_obs(curr_player_id)

            # oracle_obs = env.get_oracle_obs(curr_player_id)
            # full_obs = env.get_full_obs(curr_player_id)
            # full_obs = np.concatenate([executor_obs, oracle_obs], axis=0)

            # --------- make decision -------------

            a = np.random.choice(np.argwhere(
                valid_actions_mask).reshape([-1]))
            
            env.step(curr_player_id, a)
            
            # ------- update state encoding ------------
            
            env.render()
            te.update()
            
            for i in range(4):
                
                print("==========================================================")
                print("==================== Turn {} Player {} =====================".format(env.t.turn, i))
                print("==========================================================")
                
                print("---------- Self Info --------------")
                obs = np.array(te.self_infos[i]).reshape([18, 34]).swapaxes(0, 1)
                print(obs)
                render_encoding_self_info(obs)
                
                print("---------- Records Info --------------")
                rcd = np.array(te.records[i][-1])
                print(rcd)
                render_encoding_record(rcd)
                
                print("---------- Global Info --------------")
                gin = np.array(te.global_infos[i])
                print(gin)
                render_global_info(gin)
                
                
                # print("东风局，庄家是 {}, player {} global info".format(game % 4, i), te.global_infos[i])
            
            # break
            # print(te.records[0][-1])

        # ----------------------- get result ---------------------------------

        payoffs = np.array(env.get_payoffs())
        print("Game {}, payoffs: {}".format(game, payoffs))
        # env.render()

        success_games += 1
        game += 1

    except Exception as inst:
        game += 1
        time.sleep(0.1)
        print(
            "-------------- execption in game {} -------------------------".format(game))
        print(inst)
        env.render()
        print("-------------- replayable log -------------------------------")
        env.t.print_debug_replay()
        continue

print("Total {} random-play games, {} games without error, takes {} s".format(
    num_games, success_games, time.time() - start_time))




东风局，庄家是 0, player 0 global info [6, 7, 0, 0, 2, 1, 250, 250, 250, 250, 0, 0, 0, 0, 69]
东风局，庄家是 0, player 1 global info [6, 7, 0, 0, 3, 1, 250, 250, 250, 250, 0, 0, 0, 0, 69]
东风局，庄家是 0, player 2 global info [6, 7, 0, 0, 0, 1, 250, 250, 250, 250, 0, 0, 0, 0, 69]
东风局，庄家是 0, player 3 global info [6, 7, 0, 0, 1, 1, 250, 250, 250, 250, 0, 0, 0, 0, 69]
----------- current player: 3 -----------------
[Player 0]
Pt: 25000
Wind: West
Hand: 1m 2m 5m 5m 7m 9m 6p 6s 7s 1z 2z 5z 6z 
River: 
Riichi: No
Menzen: Yes
[Player 1]
Pt: 25000
Wind: North
Hand: 5m 6m 3p 8p 8p 8p 9p 1s 4s 7s 5z 5z 7z 
River: 
Riichi: No
Menzen: Yes
[Player 2]
Pt: 25000
Wind: East
Hand: 2m 2m 4m 4m 8m 1p 2p 3s 4s 8s 9s 9s 3z 
River: 2p1h 
Riichi: No
Menzen: Yes
[Player 3]
Pt: 25000
Wind: South
Hand: 1m 4m 7m 8m 1p 4p 5p 6p 9p 1s 9s 1z 4z 5s 
River: 
Riichi: No
Menzen: Yes
==================== Turn 3 Player 0 =====================
---------- Self Info --------------
[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 

In [ ]:
import torch
x = torch.zeros([2,4])